In [2]:
import math
import operator
import random
import re
import numpy as np


from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.stats import pearsonr
# from gensim.models import KeyedVectors
import pickle

from nltk import cluster


from nltk.cluster import cosine_distance

In [3]:
from contextlib import contextmanager
import sys, os

# To prevent page crashes

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [4]:
w_names = [
    '1800-w.npy',
    '1810-w.npy',
    '1820-w.npy',
    '1830-w.npy',
    '1840-w.npy',
    '1850-w.npy',
    '1860-w.npy',
    '1870-w.npy',
    '1880-w.npy',
    '1890-w.npy',
    '1900-w.npy',
    '1910-w.npy',
    '1920-w.npy',
    '1930-w.npy',
    '1940-w.npy',
    '1950-w.npy',
    '1960-w.npy',
    '1970-w.npy',
    '1980-w.npy',
    '1990-w.npy'
]

v_names = [
    '1800-vocab.pkl',
    '1810-vocab.pkl',
    '1820-vocab.pkl',
    '1830-vocab.pkl',
    '1840-vocab.pkl',
    '1850-vocab.pkl',
    '1860-vocab.pkl',
    '1870-vocab.pkl',
    '1880-vocab.pkl',
    '1890-vocab.pkl',
    '1900-vocab.pkl',
    '1910-vocab.pkl',
    '1920-vocab.pkl',
    '1930-vocab.pkl',
    '1940-vocab.pkl',
    '1950-vocab.pkl',
    '1960-vocab.pkl',
    '1970-vocab.pkl',
    '1980-vocab.pkl',
    '1990-vocab.pkl'
]

w_keys = [
    '1800',
    '1810',
    '1820',
    '1830',
    '1840',
    '1850',
    '1860',
    '1870',
    '1880',
    '1890',
    '1900',
    '1910',
    '1920',
    '1930',
    '1940',
    '1950',
    '1960',
    '1970',
    '1980',
    '1990'
]

In [5]:
commonvocab = {}
with open('common-vocab-index.pickle', 'rb') as f:
    commonvocab = pickle.load(f)
list(commonvocab.keys())[7260]

'palace'

In [6]:
# common word vectors
commonvec = {}
with open('common-vocab-vectors.pickle', 'rb') as f:
    commonvec = pickle.load(f)
commonvec['1990'][7260] #check if loaded correctly

array([ 1.52451825e-01, -3.24319627e-02, -2.77466886e-02,  8.13460443e-02,
        5.92073572e-02,  5.29265206e-03,  2.82709899e-02, -4.10130175e-02,
       -3.72760571e-02, -2.26741472e-02, -6.70740656e-02,  2.33680557e-02,
        1.48349215e-01,  5.86283656e-02, -4.34921838e-02,  1.52196014e-02,
       -1.02812216e-02,  4.02334360e-02, -9.89008183e-03,  1.49252177e-02,
       -4.88889459e-02,  5.11624657e-04, -8.23032937e-02, -7.03315042e-02,
        8.34581908e-03, -5.70643335e-02,  2.80378371e-02, -3.16122645e-02,
        1.04138866e-01, -2.80117488e-02, -9.80843702e-03,  4.50507456e-02,
       -7.38177866e-02,  3.43025327e-02, -2.65286786e-02, -1.89344390e-02,
       -2.94337994e-02,  1.53375506e-02, -3.19133665e-02, -5.45609911e-02,
       -8.41236687e-02, -1.08542055e-01,  3.99398344e-02, -5.94535111e-02,
       -3.84440583e-03,  7.81781597e-03, -2.59605284e-02,  5.84319028e-02,
       -5.11038999e-02,  8.68011562e-02, -4.92141797e-02,  4.78485163e-02,
       -2.52334661e-02, -

In [7]:
# clusters
clustsagg = {}
with open('cluster-center-vectors.pickle', 'rb') as f:
    clustsagg = pickle.load(f)
# check size (100 clusters of 300 dim vectors per timestamp)
clustsagg['1810'].shape

(100, 300)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
timestepSimilarities = {}
for t in w_keys:
    timestepSimilarities[t] = cosine_similarity(commonvec[t],clustsagg[t])


In [9]:
timestepSimilarities['1810'].shape

(7446, 100)

In [10]:
cnt = 0
for t in w_keys:
    for w in timestepSimilarities[t]:
        k=0
        for i in w:
            if(i != 0):
                k=1
        cnt += k
cnt

102168

In [11]:
cnt/20
#so about 2/3rd of the word had some similarities to the clusters

5108.4

In [12]:
timestepChanges = [] #row index is index of word, each column is the timestep difference

for i in range(timestepSimilarities['1810'].shape[0]):
    worddelta = []
    t1 = timestepSimilarities['1810'][i]
    for t in w_keys[1:]:
        t2 = timestepSimilarities[t][i]
        delta = t2-t1
        worddelta.append(delta)
        t1 = t2
    timestepChanges.append(worddelta)



In [13]:
len(timestepChanges[0]) # should be 19, since delta over 20 timesteps
#'{:.2e}'.format(timestepSimilarities['1800'][0][35])

19

In [14]:
# test before loop
word =  timestepSimilarities['1810'][0]
npword = word
npword.shape
clustindexes = np.argpartition(npword, -5)[-5:]
clustindexes[np.argsort(npword[clustindexes])]

array([34, 33, 35, 32, 99], dtype=int64)

In [16]:
closestClusters = []
movementAltMethod = []
topk = 5
for i in range(len(timestepSimilarities['1810'])):
    npword = timestepSimilarities['1800'][i]
    clustindexes = np.argpartition(npword, -topk)[-topk:]
    closestClusters.append(clustindexes[np.argsort(npword[clustindexes])])
    movealtword = []
    for t in range(len(timestepChanges[0])):
        word_t = timestepSimilarities[w_keys[t]][i]
        maxindex_t = np.argpartition(word_t, -topk)[-topk:]
        diff = timestepChanges[i][t]
        movealtword.append(sum(diff[maxindex_t])/len(diff[maxindex_t]))
    movementAltMethod.append(movealtword)
len(closestClusters)

7446

In [17]:
len(movementAltMethod)

7446

In [141]:
len(closestClusters[7445])

5

In [142]:
wordMovements = [] #relative to most similar clusters
for i in range(len(timestepChanges)):
    clusts = closestClusters[i]
    movement = timestepChanges[i]
    avgmovementOverTime = []
    for diff in movement:
        avgmovement = sum(diff[clusts])/len(diff[clusts])
        avgmovementOverTime.append(avgmovement)
    wordMovements.append(avgmovementOverTime)

In [143]:
len(wordMovements[0])

19

In [18]:
maxChangeMagnitude = []
timestepOfChange = []
def getMove(wordMovements):
    for wordMove in wordMovements:
        maximumChange = max(wordMove)
        maxIndex = wordMove.index(maximumChange)
        maxChangeMagnitude.append(maximumChange)
        timestepOfChange.append(maxIndex)
# getMove(wordMovements)
getMove(movementAltMethod)
len(timestepOfChange)

7446

In [58]:
len(maxChangeMagnitude)

7446

In [75]:
n = 60 # number of words for analysis
offset = len(maxChangeMagnitude)-n #start looking at words from this index
magnitudes = np.array(maxChangeMagnitude)
nMostChangedWords = []
if(offset!=0):
    nMostChangedWords = np.argsort(magnitudes)[-n-offset:-offset][::-1]
else:
    nMostChangedWords = np.argsort(magnitudes)[-n:][::-1]
nTopValues = [magnitudes[i] for i in nMostChangedWords]

In [76]:
nTopValues

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [77]:
topWords = []
timeStep = []
vocab = list(commonvocab.keys())
for i in nMostChangedWords:
    topWords.append(vocab[i])
    t = timestepOfChange[i]
    timeStep.append(w_keys[t] + ' to ' + w_keys[t+1])

In [78]:
for i in range(len(topWords)):
    print("{:20s} from {} with score {}".format(topWords[i],timeStep[i], nTopValues[i]))

angeles              from 1800 to 1810 with score 0.0
drink                from 1800 to 1810 with score 0.0
hardest              from 1800 to 1810 with score 0.0
radio                from 1800 to 1810 with score 0.0
rubbed               from 1800 to 1810 with score 0.0
goin                 from 1800 to 1810 with score 0.0
outskirts            from 1800 to 1810 with score 0.0
avenue               from 1800 to 1810 with score 0.0
eat                  from 1800 to 1810 with score 0.0
mess                 from 1800 to 1810 with score 0.0
nothin               from 1800 to 1810 with score 0.0
safe                 from 1800 to 1810 with score 0.0
tucked               from 1800 to 1810 with score 0.0
kid                  from 1800 to 1810 with score 0.0
smiling              from 1800 to 1810 with score 0.0
expect               from 1800 to 1810 with score 0.0
remembered           from 1800 to 1810 with score 0.0
landing              from 1800 to 1810 with score 0.0
uh                   from 18

In [149]:
# timeStep

['1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1830 to 1840',
 '1830 to 1840',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1820 to 1830',
 '1830 to 1840']